In [1]:
!pip --version

pip 21.0.1 from /shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/pip (python 3.8)


In [2]:
!pip install langid

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import langid
import pandas as pd
import json
from bs4 import BeautifulSoup, Comment
import requests
import threading

In [6]:
df = pd.read_csv("URL Classification.csv")

In [7]:
df.head()

,1,http://www.liquidgeneration.com/,Adult
0,2,http://www.onlineanime.org/,Adult
1,3,http://www.ceres.dti.ne.jp/~nekoi/senno/senfir...,Adult
2,4,http://www.galeon.com/kmh/,Adult
3,5,http://www.fanworkrecs.com/,Adult
4,6,http://www.animehouse.com/,Adult


In [11]:
i = df[(df.Adult=="Adult")].index

In [12]:
i

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            35314, 35315, 35316, 35317, 35318, 35319, 35320, 35321, 35322,
            35323],
           dtype='int64', length=35324)

In [13]:
df = df.drop(i)

In [14]:
df.head()

,1,http://www.liquidgeneration.com/,Adult
35324,35326,http://www.awn.com,Arts
35325,35327,http://animation.about.com/,Arts
35326,35328,http://www.toonhound.com,Arts
35327,35329,http://www.geocities.com/d-patanella/,Arts
35328,35330,http://enculturation.gmu.edu/2_1/pisters.html,Arts


In [15]:
df.Adult.unique()

array(['Arts', 'Business', 'Computers', 'Games', 'Health', 'Home', 'Kids',
       'News', 'Recreation', 'Reference', 'Science', 'Shopping',
       'Society', 'Sports'], dtype=object)

In [16]:
df.size

4582959

In [17]:
len(df.index)

1527653

In [18]:
df = df.rename(columns={'http://www.liquidgeneration.com/': "URL", "Adult": "category"})

In [19]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.9999.999 Safari/537.36',
    'Referer': 'https://example.com'
}

def extract_html_from_doc(url, category):
    try:
        response = requests.get(url, headers=headers)
        content_data = {}
        soup = BeautifulSoup(response.text, 'html.parser')
        clean_soup = get_cleaned_html(soup)
        if len(soup.prettify())>10000:
            language, _ = langid.classify(clean_soup.get_text())
            if language == 'en':
                style_tags = soup.find_all('style')
                title = soup.title.text.strip()
                if len(style_tags)>0 and title:
                    content_data['prompt'] = "create a " + category + " website for " + title
                    content_data['output'] = soup.prettify().replace('\n', '')
                    return content_data
    except:
        return None
    
def get_cleaned_html(soup):
    comments = soup.find_all(string=lambda string: isinstance(string, Comment))
    for comment in comments:
        comment.extract()
    return soup

In [20]:
df.head()

,1,URL,category
35324,35326,http://www.awn.com,Arts
35325,35327,http://animation.about.com/,Arts
35326,35328,http://www.toonhound.com,Arts
35327,35329,http://www.geocities.com/d-patanella/,Arts
35328,35330,http://enculturation.gmu.edu/2_1/pisters.html,Arts


In [21]:

def getData(df, num, f):
    output_data = []
    for d in range(num,num+1000):
        url = df.iloc[d]['URL']
        category = df.iloc[d]['category']
        output = extract_html_from_doc(url, category)
        if output != None:
            output_data.append(output)
        if d%25 == 0:
            print(d)
        json_data = json.dumps(output_data, indent=4)

    with open('data'+f+'.json', 'w') as json_file:
        json_file.write(json_data)


In [22]:
t1 = threading.Thread(target=getData, args=(df,1000000,"1",))
t2 = threading.Thread(target=getData, args=(df,1001000,"2",))
t3 = threading.Thread(target=getData, args=(df,1002000,"3",))
t4 = threading.Thread(target=getData, args=(df,1003000,"4",))
t5 = threading.Thread(target=getData, args=(df,1004000,"5",))

In [23]:
t1.start()
t2.start()
t3.start()
t4.start()
t5.start()